<a href="https://colab.research.google.com/github/around-star/Dance_Form/blob/master/Dance_Form.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Dance Form Prediction

In [1]:
import cv2 as cv
import numpy as np
import os
import tensorflow as tf
import pandas as pd 
import csv
from google.colab.patches import cv2_imshow

In [2]:
from google.colab import drive
drive.mount('/gdrive')
os.chdir('/gdrive/My Drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [3]:
files_direc="dataset/train"
test_direc="dataset/test"
df = pd.read_csv('dataset/train.csv')
test_df = pd.read_csv('dataset/test.csv')

In [4]:
df.head(10)

,Image,target
0,96.jpg,manipuri
1,163.jpg,bharatanatyam
2,450.jpg,odissi
3,219.jpg,kathakali
4,455.jpg,odissi
5,46.jpg,kathak
6,326.jpg,sattriya
7,408.jpg,kuchipudi
8,365.jpg,sattriya
9,422.jpg,kuchipudi


In [5]:
def one_hot_encode(data, val=8):
  n = len(data)
  a = np.zeros((n,val))
  a[range(n) , data] = 1
  return a

In [6]:
embed={}
embed_data={}

def data_augment(direc):

  x_train = df['Image']
  y_train = df['target']
  j=0

  x_batch = []
  y = []
  for i in y_train.unique():
    embed[i]=j
    embed_data[j]=i
    j+=1
  for k in range(len(x_train)):
    arr = cv.resize(cv.imread(os.path.join(direc , x_train[k])) , (200,200) )
    aug1 = tf.keras.preprocessing.image.random_shift(arr, 0.3, 0.5)
    aug2 = tf.image.flip_left_right(arr)
    aug3 = tf.image.adjust_brightness(arr, 0.5)
    aug4 = tf.image.flip_left_right(aug3)
    aug5 = tf.image.adjust_brightness(aug2, 0.5)
    aug =[arr,aug1, aug2, aug3, aug4, aug5]
    [x_batch.append(i) for i in aug]

    [y.append(embed[y_train[k]]) for _ in range(len(aug))]
    
  y_batch = one_hot_encode(y)
  
  x_batch = np.stack(x_batch , axis = 0)


  return x_batch , y_batch

x_full,y_full = data_augment(files_direc)

In [ ]:
np.expand_dims(x_full[0], axis=0)

In [30]:
def create_batch(batch_size=10):
  x_batch=[]
  index = np.random.randint(len(x_full) , size = batch_size)
  x_batch = [x_full[i] for i in index]
  x_batch = np.stack(x_batch, axis=0)

  y_batch = [y_full[i] for i in index]
  y_batch = np.stack(y_batch, axis=0)

  return x_batch, y_batch

In [31]:
x_check , y_check= create_batch()
y_check[0:3]

array([[0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.]])

In [10]:
def create_model(x_shape):
  pretrained_model = tf.keras.applications.ResNet50(include_top=False, input_shape=x_shape)
  for i,layer in enumerate(pretrained_model.layers):
    if(i <= 60):
      layer.trainable = False
    
  #x = tf.keras.layers.Conv2D(128, kernel_size=(7, 7), strides=(1,1), padding='valid', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(pretrained_model.output)
  #x = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='valid')(x)

  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(1024, activation='relu')(x)

  x = tf.keras.layers.Dropout(rate=0.5)(x)

  outputs = tf.keras.layers.Dense(8)(x)

  model = tf.keras.Model(inputs=pretrained_model.input , outputs=outputs)

  return model


model= create_model(x_shape = x_full[0].shape)

94773248/94765736 [==============================] - 1s 0us/step


In [11]:
def compute_loss(y_pred , y_true):
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_true, y_pred))
  return loss

In [12]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
def train_step():
  with tf.GradientTape() as tape:
    x,y = create_batch(10)
    y_pred = model(x)

    loss = compute_loss(y_pred , y)

  grads = tape.gradient(loss , model.variables)
  
  optimizer.apply_gradients(zip(grads , model.variables))


In [ ]:
iteration = 100
for iterate in range(iteration):
  train_step()


In [56]:
def predict(x):
  y_predict = model(x)
  y_predict = np.argmax(y_predict, axis=1)

  return y_predict

print(embed_data[predict(np.expand_dims(x_check[6], axis=0))[0]])

odissi


In [55]:
embed_data[np.argmax(y_check[6], axis=-1)]

'odissi'

##Storing the test results in a csv file

In [ ]:
def test(direc):
  img_data = test_df['Image']
  with open('result.csv', 'w', newline='') as f:
    thewriter = csv.writer(f)

    for i in img_data:
      img = cv.resize(cv.imread(os.path.join(direc,i)) , (200,200))

      y_pred = predict(np.expand_dims(img, axis=0))


      thewriter.writerow([i, y_pred])

test(test_direc)